In [44]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt

<h1>Theory</h1>
<h3>Note</h3>
<ul>
    <li>The explanation below belongs to the book Computer Vision: Algorithms and Applications by Richard Szeliski</li>
</ul>
<br />
<h3>Image Processing</h3>
<ul>
    <li>A general image processing operator is a function that takes one or more input images and produces an output image.</li>
    <li>Image transforms can be seen as:</li>
    <li>
        <ul>
            <li>Point operators (pixel transforms)</li>
            <li>Neighborhood (area-based) operators</li>
        </ul>
    </li>
</ul>
<br />
<h3>Pixel Transforms</h3>
<ul>
    <li>In this kind of image processing transform, each output pixel's value depends on only the corresponding input pixel value (plus, potentially, some globally collected information or parameters).</li>
    <li>Examples of such operators include brightness and contrast adjustments as well as color correction and transformations.</li>
</ul>
<br />
<h3>Brightness and contrast adjustments</h3>
<ul>
    <li>Two commonly used point processes are multiplication and addition with a constant:</li>
    <h4>g(x)=αf(x)+β</h4>
    <li>The parameters α>0 and β are often called the gain and bias parameters; sometimes these parameters are said to control contrast and brightness respectively.</li>
    <li>You can think of f(x) as the source image pixels and g(x) as the output image pixels. Then, more conveniently we can write the expression as:
        <br /><h4>g(i,j)=α⋅f(i,j)+β</h4><br />   
        where i and j indicates that the pixel is located in the i-th row and j-th column.</li>
</ul>


In [45]:
# Enhance  Image using PIL

image = Image.open('p1/p3.bmp')
image = image.resize((200, 200))
contrast = -1.5
bright = 1
sharpness = 10

cont_image = ImageEnhance.Contrast(image)
new_image = cont_image .enhance(contrast)

bright_image = ImageEnhance.Brightness(new_image)
new_image = bright_image.enhance(bright)

sharp_image = ImageEnhance.Sharpness(new_image)
new_image = sharp_image .enhance(sharpness)

new_image.show()
new_image.save('./all_image/image.jpg')

<h3>Binarization</h3>


In [46]:

import cv2
image = cv2.imread('./all_image/image.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.Sobel(image,cv2.CV_64F,dx=0,dy=1,ksize=5)
image = cv2.blur(image, (5, 5))

ret, img = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY)

cv2.imwrite('./all_image/th_image.jpg', img)
# print(ret)
cv2.imshow('Thresold',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<h3>Ridge Extraction</h3>


In [47]:
from skimage import io, filters, feature
from skimage.color import rgb2gray

from skimage.filters import  meijering, sato, frangi, hessian

image = cv2.imread("./all_image/th_image.jpg")
img =   rgb2gray(image)

meijering_img = meijering(img)
sato_img = sato(img)
hessian_img = hessian(img)


cv2.imwrite('./all_image/meijering_img.jpg',meijering_img)
cv2.imwrite('./all_image/sato_img.jpg',sato_img)
cv2.imwrite('./all_image/hessian_img.jpg',hessian_img)

cv2.imshow('Meijering Image',meijering_img)
cv2.imshow('Sato Image',sato_img)
cv2.imshow('Hessian Image',hessian_img)
cv2.waitKey(0)
cv2.destroyAllWindows()



<h3>Minutiae Point Extraction Algorithm</h3>


In [48]:
import skimage.morphology
from skimage.morphology import convex_hull_image, erosion
from skimage.morphology import square
import math

class MinutiaeFeature(object):
    def __init__(self, locX, locY, Orientation, Type):
        self.locX = locX
        self.locY = locY
        self.Orientation = Orientation
        self.Type = Type

class FingerprintFeatureExtractor(object):
    def __init__(self):
        self._mask = []
        self._skel = []
        self.minutiaeTerm = []
        self.minutiaeBif = []
        self._spuriousMinutiaeThresh = 10

    def setSpuriousMinutiaeThresh(self, spuriousMinutiaeThresh):
        self._spuriousMinutiaeThresh = spuriousMinutiaeThresh

    def __skeletonize(self, img):
        img = np.uint8(img > 128)
        self._skel = skimage.morphology.skeletonize(img)
        self._skel = np.uint8(self._skel) * 255
        self._mask = img * 255

    def __computeAngle(self, block, minutiaeType):
        angle = []
        (blkRows, blkCols) = np.shape(block)
        CenterX, CenterY = (blkRows - 1) / 2, (blkCols - 1) / 2
        if (minutiaeType.lower() == 'termination'):
            sumVal = 0
            for i in range(blkRows):
                for j in range(blkCols):
                    if ((i == 0 or i == blkRows - 1 or j == 0 or j == blkCols - 1) and block[i][j] != 0):
                        angle.append(-math.degrees(math.atan2(i - CenterY, j - CenterX)))
                        sumVal += 1
                        if (sumVal > 1):
                            angle.append(float('nan'))
            return (angle)

        elif (minutiaeType.lower() == 'bifurcation'):
            (blkRows, blkCols) = np.shape(block)
            CenterX, CenterY = (blkRows - 1) / 2, (blkCols - 1) / 2
            angle = []
            sumVal = 0
            for i in range(blkRows):
                for j in range(blkCols):
                    if ((i == 0 or i == blkRows - 1 or j == 0 or j == blkCols - 1) and block[i][j] != 0):
                        angle.append(-math.degrees(math.atan2(i - CenterY, j - CenterX)))
                        sumVal += 1
            if (sumVal != 3):
                angle.append(float('nan'))
            return (angle)

    def __getTerminationBifurcation(self):
        self._skel = self._skel == 255
        (rows, cols) = self._skel.shape
        self.minutiaeTerm = np.zeros(self._skel.shape)
        self.minutiaeBif = np.zeros(self._skel.shape)

        for i in range(1, rows - 1):
            for j in range(1, cols - 1):
                if (self._skel[i][j] == 1):
                    block = self._skel[i - 1:i + 2, j - 1:j + 2]
                    block_val = np.sum(block)
                    if (block_val == 2):
                        self.minutiaeTerm[i, j] = 1
                    elif (block_val == 4):
                        self.minutiaeBif[i, j] = 1

        self._mask = convex_hull_image(self._mask > 0)
        self._mask = erosion(self._mask, square(5))  # Structuing element for mask erosion = square(5)
        self.minutiaeTerm = np.uint8(self._mask) * self.minutiaeTerm

    def __removeSpuriousMinutiae(self, minutiaeList, img):
        img = img * 0
        SpuriousMin = []
        numPoints = len(minutiaeList)
        D = np.zeros((numPoints, numPoints))
        for i in range(1,numPoints):
            for j in range(0, i):
                (X1,Y1) = minutiaeList[i]['centroid']
                (X2,Y2) = minutiaeList[j]['centroid']

                dist = np.sqrt((X2-X1)**2 + (Y2-Y1)**2)
                D[i][j] = dist
                if(dist < self._spuriousMinutiaeThresh):
                    SpuriousMin.append(i)
                    SpuriousMin.append(j)

        SpuriousMin = np.unique(SpuriousMin)
        for i in range(0,numPoints):
            if(not i in SpuriousMin):
                (X,Y) = np.int16(minutiaeList[i]['centroid'])
                img[X,Y] = 1

        img = np.uint8(img)
        return(img)

    def __cleanMinutiae(self, img):
        self.minutiaeTerm = skimage.measure.label(self.minutiaeTerm, connectivity=2)
        RP = skimage.measure.regionprops(self.minutiaeTerm)
        self.minutiaeTerm = self.__removeSpuriousMinutiae(RP, np.uint8(img))

    def __performFeatureExtraction(self):
        FeaturesTerm = []
        self.minutiaeTerm = skimage.measure.label(self.minutiaeTerm, connectivity=2)
        RP = skimage.measure.regionprops(np.uint8(self.minutiaeTerm))

        WindowSize = 2  # --> For Termination, the block size must can be 3x3, or 5x5. Hence the window selected is 1 or 2
        FeaturesTerm = []
        for num, i in enumerate(RP):
            (row, col) = np.int16(np.round(i['Centroid']))
            block = self._skel[row - WindowSize:row + WindowSize + 1, col - WindowSize:col + WindowSize + 1]
            angle = self.__computeAngle(block, 'Termination')
            if(len(angle) == 1):
                FeaturesTerm.append(MinutiaeFeature(row, col, angle, 'Termination'))

        FeaturesBif = []
        self.minutiaeBif = skimage.measure.label(self.minutiaeBif, connectivity=2)
        RP = skimage.measure.regionprops(np.uint8(self.minutiaeBif))
        WindowSize = 1  # --> For Bifurcation, the block size must be 3x3. Hence the window selected is 1
        for i in RP:
            (row, col) = np.int16(np.round(i['Centroid']))
            block = self._skel[row - WindowSize:row + WindowSize + 1, col - WindowSize:col + WindowSize + 1]
            angle = self.__computeAngle(block, 'Bifurcation')
            if(len(angle) == 3):
                FeaturesBif.append(MinutiaeFeature(row, col, angle, 'Bifurcation'))
        return (FeaturesTerm, FeaturesBif)

    def extractMinutiaeFeatures(self, img):
        self.__skeletonize(img)

        self.__getTerminationBifurcation()

        self.__cleanMinutiae(img)

        FeaturesTerm, FeaturesBif = self.__performFeatureExtraction()
        return(FeaturesTerm, FeaturesBif)

    def showResults(self, FeaturesTerm, FeaturesBif):
        
        (rows, cols) = self._skel.shape
        DispImg = np.zeros((rows, cols, 3), np.uint8)
        DispImg[:, :, 0] = 255*self._skel
        DispImg[:, :, 1] = 255*self._skel
        DispImg[:, :, 2] = 255*self._skel

        for idx, curr_minutiae in enumerate(FeaturesTerm):
            row, col = curr_minutiae.locX, curr_minutiae.locY
            (rr, cc) = skimage.draw.circle_perimeter(row, col, 3)
            skimage.draw.set_color(DispImg, (rr, cc), (0, 0, 255))

        for idx, curr_minutiae in enumerate(FeaturesBif):
            row, col = curr_minutiae.locX, curr_minutiae.locY
            (rr, cc) = skimage.draw.circle_perimeter(row, col, 3)
            skimage.draw.set_color(DispImg, (rr, cc), (255, 0, 0))
        
        cv2.imshow('output', DispImg)
        cv2.waitKey(0)

    def saveResult(self, FeaturesTerm, FeaturesBif):
        (rows, cols) = self._skel.shape
        DispImg = np.zeros((rows, cols, 3), np.uint8)
        DispImg[:, :, 0] = 255 * self._skel
        DispImg[:, :, 1] = 255 * self._skel
        DispImg[:, :, 2] = 255 * self._skel

        for idx, curr_minutiae in enumerate(FeaturesTerm):
            row, col = curr_minutiae.locX, curr_minutiae.locY
            (rr, cc) = skimage.draw.circle_perimeter(row, col, 3)
            skimage.draw.set_color(DispImg, (rr, cc), (0, 0, 255))

        for idx, curr_minutiae in enumerate(FeaturesBif):
            row, col = curr_minutiae.locX, curr_minutiae.locY
            (rr, cc) = skimage.draw.circle_perimeter(row, col, 3)
            skimage.draw.set_color(DispImg, (rr, cc), (255, 0, 0))
        cv2.imwrite('result/result.jpg', DispImg)

def extract_minutiae_features(img, spuriousMinutiaeThresh=10, invertImage=False, showResult=False, saveResult=False):
    feature_extractor = FingerprintFeatureExtractor()
    feature_extractor.setSpuriousMinutiaeThresh(spuriousMinutiaeThresh)
    if (invertImage):
        img = 255 - img;

    FeaturesTerm, FeaturesBif = feature_extractor.extractMinutiaeFeatures(img)

    if (saveResult):
        feature_extractor.saveResult(FeaturesTerm, FeaturesBif)

    if(showResult):
        feature_extractor.showResults(FeaturesTerm, FeaturesBif)

    return(FeaturesTerm, FeaturesBif)

In [49]:
img = cv2.imread('all_image/th_image.jpg',0)
terminations, bifurcations = extract_minutiae_features(img,spuriousMinutiaeThresh=10, invertImage = False, showResult=True, saveResult = True)
print(terminations)

[<__main__.MinutiaeFeature object at 0x0000029FD29CDB20>, <__main__.MinutiaeFeature object at 0x0000029FC04F02E0>, <__main__.MinutiaeFeature object at 0x0000029FC04F05E0>, <__main__.MinutiaeFeature object at 0x0000029FC04F05B0>, <__main__.MinutiaeFeature object at 0x0000029FC04F0610>, <__main__.MinutiaeFeature object at 0x0000029FC04F06A0>, <__main__.MinutiaeFeature object at 0x0000029FC04F06D0>, <__main__.MinutiaeFeature object at 0x0000029FC04F0430>, <__main__.MinutiaeFeature object at 0x0000029FC04F0190>, <__main__.MinutiaeFeature object at 0x0000029FC04F07C0>, <__main__.MinutiaeFeature object at 0x0000029FC04F0640>]


<h3>Matching</h3>


In [50]:
import os
import cv2

sample = cv2.imread(
    "./all_image/th_image.jpg"
)

best_score = counter = 0
filename = image = kp1 = kp2 = mp = None
for file in os.listdir(
    "./test_db/"
):
    
    fingerprint_img = cv2.imread(
        "./test_db/" + file
    )
    sift = cv2.SIFT_create()
    keypoints_1, des1 = sift.detectAndCompute(sample, None)
    keypoints_2, des2 = sift.detectAndCompute(fingerprint_img, None)
    matches = cv2.FlannBasedMatcher({"algorithm": 1, "trees": 10}, {}).knnMatch(
        des1, des2, k=2
    )

    match_points = []
    for p, q in matches:
        if p.distance < 0.1 * q.distance:
            match_points.append(p)

    keypoints = 0
    if len(keypoints_1) <= len(keypoints_2):
        keypoints = len(keypoints_1)
    else:
        keypoints = len(keypoints_2)
    if len(match_points) / keypoints * 100 > best_score:
        best_score = len(match_points) / keypoints * 100
        filename = file
        image = fingerprint_img
        kp1, kp2, mp = keypoints_1, keypoints_2, match_points

print("Best match:  " + filename)
print("Best score:  " + str(best_score))

# if len(match_points) > 0:
result = cv2.drawMatches(sample, kp1, image, kp2, mp, None)
# result = cv2.resize(result, None, fx=5, fy=5)
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Best match:  th_image.jpg
Best score:  100.0
